In [5]:
import datetime as dt
import pandas as pd
import requests
from dask.distributed import LocalCluster
from dask.distributed import Client

In [23]:
api = 'http://172.16.0.170:5000/stream'

def worker(api, query):
    start = dt.datetime.now()
    resp = requests.post(api, data={'s': query})
    elapsed = dt.datetime.now() - start
    return elapsed.total_seconds() * 1000, resp.status_code

In [24]:
configs = [(1, 1), (1, 2), (1, 4), (2, 4), (4, 4), (4, 8), (4, 16), (8, 16), (8, 32), (8, 64)]
num_q = 4096

In [25]:
config_res = []

for config in configs:
    c = LocalCluster(*config)
    client = Client(c)

    qs = ['你好'] * num_q

    start = dt.datetime.now()
    results = []

    for s in qs:
        results.append(client.submit(worker, api, s, pure=False))

    results = client.gather(results)
    elapsed = (dt.datetime.now() - start).total_seconds() * 1000

    client.close()
    c.close()

    times = [r for r, _ in results]
    status = [1 if r == 200 else 0 for _, r in results]
    df = pd.DataFrame({'latency': times, 'status': status})
    res = df[['latency']].describe()
    res.loc['QPS', 'latency'] = len(qs) / elapsed * 1000
    res.loc['success', 'latency'] = df['status'].sum() / len(df)
    config_res.append(res)
    
total_df = pd.concat(config_res, axis=1)

In [26]:
total_df

,latency,latency,latency,latency,latency,latency,latency,latency,latency,latency
count,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000
mean,120.887207,121.596774,126.088510,130.022172,137.341852,156.942026,141.123642,202.122946,291.681502,456.658928
std,99.807526,10.471394,11.963108,10.266972,10.561385,18.814446,55.003492,77.447668,131.400309,172.883442
min,13.121000,14.163000,15.993000,16.515000,17.426000,28.772000,32.289000,16.427000,27.625000,32.829000
25%,119.010500,120.339000,122.531750,125.149500,131.233750,145.593750,98.161250,154.496250,201.124250,371.781500
50%,120.126000,122.202000,126.213000,129.440000,137.397500,157.650500,129.205500,194.999500,275.766000,459.430000
75%,120.990000,124.453750,130.513000,134.520250,143.187250,169.167250,185.662500,235.992250,372.455000,532.817750
max,6480.732000,186.229000,178.819000,187.413000,194.583000,223.372000,276.593000,397.135000,1467.962000,1756.137000
QPS,8.169781,16.156716,30.755261,59.793799,111.478979,190.468529,408.656208,544.231209,703.889280,854.469778
success,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
